In [3]:
import pandas as pd
import numpy as np
import random
from datetime import date, timedelta
import os

# ── SET SEED for reproducibility ──────────────────────────────
np.random.seed(42)
random.seed(42)

# ── CREATE OUTPUT FOLDER ──────────────────────────────────────
os.makedirs('data/generated', exist_ok=True)

print("Generating synthetic retail data...")

# ════════════════════════════════════════════
# 1. DIM_DATE
# ════════════════════════════════════════════
start_date = date(2024, 1, 1)
end_date   = date(2025, 12, 31)

dates = []
current = start_date
fiscal_week = 1
while current <= end_date:
    is_weekend = current.weekday() >= 5
    # Reset fiscal week at start of each year
    if current.month == 1 and current.day <= 7 and current.weekday() == 6:
        fiscal_week = 1
    elif current.weekday() == 6:
        fiscal_week += 1
    
    dates.append({
        'date_id':        int(current.strftime('%Y%m%d')),
        'full_date':      current.strftime('%Y-%m-%d'),
        'day_of_week':    current.strftime('%A'),
        'day_number':     current.weekday() + 1,
        'week_number':    current.isocalendar()[1],
        'fiscal_week':    fiscal_week,
        'month_number':   current.month,
        'month_name':     current.strftime('%B'),
        'quarter':        (current.month - 1) // 3 + 1,
        'year':           current.year,
        'is_weekend':     is_weekend,
        'is_bank_holiday': current in [
            date(2024,1,1), date(2024,3,29), date(2024,4,1),
            date(2024,5,6), date(2024,5,27), date(2024,8,26),
            date(2024,12,25), date(2024,12,26),
            date(2025,1,1), date(2025,4,18), date(2025,4,21),
            date(2025,5,5), date(2025,5,26), date(2025,8,25),
            date(2025,12,25), date(2025,12,26),
        ]
    })
    current += timedelta(days=1)

dim_date = pd.DataFrame(dates)
dim_date.to_csv('data/generated/dim_date.csv', index=False)
print(f"✅ dim_date: {len(dim_date)} rows")

# ════════════════════════════════════════════
# 2. DIM_STORE
# ════════════════════════════════════════════
stores = [
    (1, 'London Islington',   'London',   'Standard',    4200, 'manager1@retail.com'),
    (2, 'London Hackney',     'London',   'Express',     2100, 'manager2@retail.com'),
    (3, 'Manchester Central', 'North',    'Superstore',  8500, 'manager3@retail.com'),
    (4, 'Manchester Didsbury','North',    'Standard',    3800, 'manager4@retail.com'),
    (5, 'Birmingham City',    'Midlands', 'Superstore',  9200, 'manager5@retail.com'),
    (6, 'Birmingham Moseley', 'Midlands', 'Standard',    3500, 'manager6@retail.com'),
    (7, 'Cardiff Central',    'Wales',    'Standard',    4100, 'manager7@retail.com'),
    (8, 'Swansea',            'Wales',    'Express',     2300, 'manager8@retail.com'),
    (9, 'Bristol Clifton',    'South',    'Standard',    3900, 'manager9@retail.com'),
    (10,'Southampton',        'South',    'Express',     2600, 'manager10@retail.com'),
]
dim_store = pd.DataFrame(stores, columns=[
    'store_id','store_name','region','store_format',
    'store_size_sqft','manager_email'
])
dim_store['open_date'] = '2020-01-01'
dim_store.to_csv('data/generated/dim_store.csv', index=False)
print(f"✅ dim_store: {len(dim_store)} rows")

# ════════════════════════════════════════════
# 3. DIM_PRODUCT
# ════════════════════════════════════════════
categories = {
    'Ambient':  ['Pasta','Rice','Tinned Tomatoes','Cereal','Coffee','Tea','Biscuits','Crisps','Cooking Oil','Soup'],
    'Fresh':    ['Milk','Bread','Cheese','Yoghurt','Butter','Eggs','Salad','Chicken','Beef Mince','Salmon'],
    'Frozen':   ['Chips','Pizza','Ice Cream','Peas','Fish Fingers','Ready Meals','Burgers','Waffles'],
    'BWS':      ['Lager 4pk','Wine Red','Wine White','Spirits Vodka','Cider 4pk','Prosecco','Ale 4pk'],
    'Non-Food': ['Washing Powder','Shampoo','Toilet Roll','Kitchen Roll','Bin Bags','Toothpaste','Batteries'],
}

products = []
product_id = 1
for category, items in categories.items():
    for i, item in enumerate(items):
        cost = round(random.uniform(0.30, 8.50), 2)
        products.append({
            'product_id':   product_id,
            'product_name': item,
            'category':     category,
            'subcategory':  category + '_sub' + str(i % 3 + 1),
            'unit_cost':    cost,
            'is_active':    True
        })
        product_id += 1

dim_product = pd.DataFrame(products)
dim_product.to_csv('data/generated/dim_product.csv', index=False)
print(f"✅ dim_product: {len(dim_product)} rows")

# ════════════════════════════════════════════
# 4. FACT_TRANSACTIONS (200K+ rows)
# ════════════════════════════════════════════
# Price lookup
price_map = {
    'Ambient': (0.80, 4.50), 'Fresh': (0.60, 6.00),
    'Frozen':  (1.20, 5.50), 'BWS':   (3.50, 18.00),
    'Non-Food':(1.00, 7.00)
}

transactions = []
transaction_id = 1

# Trading days only
trading_days = dim_date[
    (dim_date['is_weekend'] == False) & 
    (dim_date['is_bank_holiday'] == False)
]['date_id'].tolist()

# Weekend days too (retail trades weekends)
all_days = dim_date['date_id'].tolist()

for date_id in all_days:
    row = dim_date[dim_date['date_id'] == date_id].iloc[0]
    is_weekend = row['is_weekend']
    month = row['month_number']
    
    # Seasonal multiplier — Christmas uplift, summer dip
    if month == 12: seasonal = 1.4
    elif month in [11, 1]: seasonal = 1.1
    elif month in [6, 7, 8]: seasonal = 0.9
    else: seasonal = 1.0
    
    # Weekend uplift
    day_multiplier = 1.3 if is_weekend else 1.0
    
    for store_id in range(1, 11):
        # Bigger stores get more transactions
        store_size = dim_store[dim_store['store_id'] == store_id]['store_size_sqft'].values[0]
        base_txns = int(store_size / 50)  # ~40-180 transactions per day
        n_transactions = int(base_txns * seasonal * day_multiplier * random.uniform(0.85, 1.15))
        
        for _ in range(n_transactions):
            # Pick random product weighted toward Fresh and Ambient
            product = dim_product.sample(1, weights=[3,3,3,2,2,2,2,1,1,1,
                                                      3,3,2,2,2,2,2,2,2,2,
                                                      2,2,2,2,2,2,2,2,
                                                      1,1,1,1,1,1,1,
                                                      1,1,1,1,1,1,1]).iloc[0]
            
            cat = product['category']
            min_p, max_p = price_map[cat]
            unit_price = round(random.uniform(min_p, max_p), 2)
            quantity = random.choices([1,2,3,4,5], weights=[50,25,12,8,5])[0]
            
            # Peak hours — morning rush and lunch
            hour = random.choices(
                list(range(7, 22)),
                weights=[5,8,12,10,8,15,12,10,8,6,5,5,4,3,2]
            )[0]
            
            transactions.append({
                'transaction_id':  transaction_id,
                'date_id':         date_id,
                'store_id':        store_id,
                'product_id':      int(product['product_id']),
                'quantity':        quantity,
                'unit_price':      unit_price,
                'transaction_hour':hour,
                'is_promotion':    random.random() < 0.12  # 12% on promotion
            })
            transaction_id += 1

fact_transactions = pd.DataFrame(transactions)
fact_transactions.to_csv('data/generated/fact_transactions.csv', index=False)
print(f"✅ fact_transactions: {len(fact_transactions):,} rows")

# ════════════════════════════════════════════
# 5. FACT_LABOUR
# ════════════════════════════════════════════
labour = []
labour_id = 1

for date_id in all_days:
    for store_id in range(1, 11):
        store_size = dim_store[dim_store['store_id'] == store_id]['store_size_sqft'].values[0]
        
        # Base hours proportional to store size
        base_hours = store_size / 500  # 4-18 hours
        row = dim_date[dim_date['date_id'] == date_id].iloc[0]
        
        # Weekend needs more staff
        if row['is_weekend']:
            hours_scheduled = round(base_hours * 1.3, 1)
        else:
            hours_scheduled = round(base_hours, 1)
        
        # Actual hours vary slightly
        hours_worked = round(hours_scheduled * random.uniform(0.90, 1.05), 1)
        hourly_rate  = round(random.uniform(11.44, 14.50), 2)  # UK NMW to £14.50
        
        labour.append({
            'labour_id':         labour_id,
            'date_id':           date_id,
            'store_id':          store_id,
            'hours_scheduled':   hours_scheduled,
            'hours_worked':      hours_worked,
            'hourly_rate':       hourly_rate,
            'total_labour_cost': round(hours_worked * hourly_rate, 2),
            'headcount':         max(1, int(hours_worked / 8))
        })
        labour_id += 1

fact_labour = pd.DataFrame(labour)
fact_labour.to_csv('data/generated/fact_labour.csv', index=False)
print(f"✅ fact_labour: {len(fact_labour):,} rows")

# ════════════════════════════════════════════
# 6. FACT_SHRINKAGE (with deliberate anomalies for Z-score demo)
# ════════════════════════════════════════════
cause_codes   = ['THEFT', 'WASTE', 'DAMAGE', 'ADMIN_ERROR']
detected_by   = ['STOCKTAKE', 'CCTV', 'MANAGER', 'SYSTEM']

shrinkage = []
shrinkage_id = 1

# Get weekly grouping
dim_date['full_date_dt'] = pd.to_datetime(dim_date['full_date'])

for _, week_row in dim_date.groupby(['year','fiscal_week']).first().reset_index().iterrows():
    fiscal_week = week_row['fiscal_week']
    year        = week_row['year']
    date_id     = int(week_row['date_id'])
    
    for store_id in range(1, 11):
        store_size = dim_store[dim_store['store_id'] == store_id]['store_size_sqft'].values[0]
        
        # Normal shrinkage: ~0.8-1.2% of estimated weekly sales
        base_shrinkage = store_size * random.uniform(0.8, 1.5)
        
        # Inject anomalies into specific stores/weeks for demo purposes
        # Store 3 (Manchester) has a theft spike in fiscal week 20
        if store_id == 3 and fiscal_week == 20:
            base_shrinkage *= 4.5  # Major anomaly — Z-score will catch this
        # Store 7 (Cardiff) has a waste spike in December
        if store_id == 7 and year == 2024 and fiscal_week >= 48:
            base_shrinkage *= 2.8
        
        # Generate 3-8 incidents per store per week
        n_incidents = random.randint(3, 8)
        incident_values = np.random.dirichlet(np.ones(n_incidents)) * base_shrinkage
        
        for val in incident_values:
            product = dim_product.sample(1).iloc[0]
            unit_cost = product['unit_cost']
            units = max(1, int(val / max(unit_cost, 0.1)))
            
            shrinkage.append({
                'shrinkage_id':    shrinkage_id,
                'date_id':         date_id,
                'store_id':        store_id,
                'product_id':      int(product['product_id']),
                'shrinkage_value': round(val, 2),
                'shrinkage_units': units,
                'cause_code':      random.choices(cause_codes, weights=[40,30,20,10])[0],
                'detected_by':     random.choices(detected_by, weights=[35,25,25,15])[0],
            })
            shrinkage_id += 1

fact_shrinkage = pd.DataFrame(shrinkage)
fact_shrinkage.to_csv('data/generated/fact_shrinkage.csv', index=False)
print(f"✅ fact_shrinkage: {len(fact_shrinkage):,} rows")

# ════════════════════════════════════════════
# SUMMARY
# ════════════════════════════════════════════
print("\n" + "="*50)
print("DATA GENERATION COMPLETE")
print("="*50)
print(f"dim_date:          {len(dim_date):>8,} rows")
print(f"dim_store:         {len(dim_store):>8,} rows")
print(f"dim_product:       {len(dim_product):>8,} rows")
print(f"fact_transactions: {len(fact_transactions):>8,} rows")
print(f"fact_labour:       {len(fact_labour):>8,} rows")
print(f"fact_shrinkage:    {len(fact_shrinkage):>8,} rows")
print(f"\nFiles saved to: data/generated/")
print("Next step: run 05_load_to_sqlite.py to load into SQL database")



Generating synthetic retail data...
✅ dim_date: 731 rows
✅ dim_store: 10 rows
✅ dim_product: 42 rows
✅ fact_transactions: 715,536 rows
✅ fact_labour: 7,310 rows
✅ fact_shrinkage: 5,697 rows

DATA GENERATION COMPLETE
dim_date:               731 rows
dim_store:               10 rows
dim_product:             42 rows
fact_transactions:  715,536 rows
fact_labour:          7,310 rows
fact_shrinkage:       5,697 rows

Files saved to: data/generated/
Next step: run 05_load_to_sqlite.py to load into SQL database


In [4]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('data/retail_kpi.db')

tables = {
    'dim_date':          'data/generated/dim_date.csv',
    'dim_store':         'data/generated/dim_store.csv',
    'dim_product':       'data/generated/dim_product.csv',
    'fact_transactions': 'data/generated/fact_transactions.csv',
    'fact_labour':       'data/generated/fact_labour.csv',
    'fact_shrinkage':    'data/generated/fact_shrinkage.csv',
}

for table_name, filepath in tables.items():
    df = pd.read_csv(filepath)
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    print(f"✅ Loaded {table_name}: {len(df):,} rows")

conn.close()
print("\n✅ Database saved: data/retail_kpi.db")

✅ Loaded dim_date: 731 rows
✅ Loaded dim_store: 10 rows
✅ Loaded dim_product: 42 rows
✅ Loaded fact_transactions: 715,536 rows
✅ Loaded fact_labour: 7,310 rows
✅ Loaded fact_shrinkage: 5,697 rows

✅ Database saved: data/retail_kpi.db


In [5]:
import sqlite3
import pandas as pd

def run_sql(query, db_path='data/retail_kpi.db'):
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

# Quick sanity check
run_sql("""
    SELECT 
        COUNT(*) as total_transactions,
        COUNT(DISTINCT store_id) as stores,
        COUNT(DISTINCT product_id) as products,
        ROUND(SUM(quantity * unit_price), 2) as total_sales_£,
        MIN(date_id) as from_date,
        MAX(date_id) as to_date
    FROM fact_transactions
""")

,total_transactions,stores,products,total_sales_£,from_date,to_date
0,715536,10,42,5422618.16,20240101,20251231


In [6]:
# Diagnose the issue
run_sql("""
    SELECT 
        MIN(unit_price) as min_price,
        MAX(unit_price) as max_price,
        AVG(unit_price) as avg_price,
        MIN(quantity) as min_qty,
        MAX(quantity) as max_qty,
        ROUND(AVG(quantity * unit_price), 2) as avg_transaction_value
    FROM fact_transactions
""")

,min_price,max_price,avg_price,min_qty,max_qty,avg_transaction_value
0,0.6,18.0,3.928727,1,5,7.58


In [7]:
# Check actual total properly
run_sql("""
    SELECT 
        COUNT(*) as transactions,
        ROUND(SUM(quantity * unit_price), 2) as total_sales,
        ROUND(AVG(quantity * unit_price), 2) as avg_basket,
        ROUND(SUM(quantity * unit_price) / 1000000, 3) as total_sales_millions
    FROM fact_transactions
""")

,transactions,total_sales,avg_basket,total_sales_millions
0,715536,5422618.16,7.58,5.423


In [8]:
run_sql("""
    WITH weekly_sales AS (
        SELECT dd.fiscal_week, dd.year, ft.store_id,
               SUM(ft.quantity * ft.unit_price) AS net_sales
        FROM fact_transactions ft
        JOIN dim_date dd ON ft.date_id = dd.date_id
        GROUP BY dd.fiscal_week, dd.year, ft.store_id
    ),
    weekly_labour AS (
        SELECT dd.fiscal_week, dd.year, fl.store_id,
               SUM(fl.total_labour_cost) AS labour_cost
        FROM fact_labour fl
        JOIN dim_date dd ON fl.date_id = dd.date_id
        GROUP BY dd.fiscal_week, dd.year, fl.store_id
    )
    SELECT 
        ws.fiscal_week,
        ds.store_name,
        ds.region,
        ROUND(ws.net_sales, 2)                         AS net_sales_£,
        ROUND(wl.labour_cost, 2)                       AS labour_cost_£,
        ROUND(wl.labour_cost / ws.net_sales * 100, 2) AS labour_pct,
        CASE 
            WHEN wl.labour_cost / ws.net_sales > 0.14 THEN '🔴 OVERSPEND'
            WHEN wl.labour_cost / ws.net_sales < 0.08 THEN '🟡 UNDERSTAFFED'
            ELSE '🟢 ON TARGET'
        END AS status
    FROM weekly_sales ws
    JOIN weekly_labour wl ON ws.fiscal_week = wl.fiscal_week 
        AND ws.year = wl.year 
        AND ws.store_id = wl.store_id
    JOIN dim_store ds ON ws.store_id = ds.store_id
    WHERE ws.year = 2024 AND ws.fiscal_week <= 8
    ORDER BY labour_pct DESC
""")

,fiscal_week,store_name,region,net_sales_£,labour_cost_£,labour_pct,status
0,8,Southampton,South,2656.62,516.82,19.45,🔴 OVERSPEND
1,8,Birmingham Moseley,Midlands,3641.66,692.22,19.01,🔴 OVERSPEND
2,5,Swansea,Wales,2394.52,443.90,18.54,🔴 OVERSPEND
3,6,Cardiff Central,Wales,4110.75,757.21,18.42,🔴 OVERSPEND
4,8,London Islington,London,4558.47,836.01,18.34,🔴 OVERSPEND
...,...,...,...,...,...,...,...
75,2,Manchester Central,North,11103.31,1567.97,14.12,🔴 OVERSPEND
76,4,Southampton,South,3449.74,486.87,14.11,🔴 OVERSPEND
77,5,London Hackney,London,2762.90,389.22,14.09,🔴 OVERSPEND
78,4,London Islington,London,5415.94,759.98,14.03,🔴 OVERSPEND


In [9]:
result = run_sql("""
    WITH weekly_shrink AS (
        SELECT 
            dd.fiscal_week, dd.year,
            fs.store_id, ds.store_name, ds.region,
            SUM(fs.shrinkage_value) AS shrinkage
        FROM fact_shrinkage fs
        JOIN dim_date  dd ON fs.date_id  = dd.date_id
        JOIN dim_store ds ON fs.store_id = ds.store_id
        GROUP BY dd.fiscal_week, dd.year, fs.store_id, ds.store_name, ds.region
    ),
    store_stats AS (
        SELECT 
            store_id,
            AVG(shrinkage)                                    AS avg_shrinkage,
            SQRT(AVG(shrinkage * shrinkage) - 
                 AVG(shrinkage) * AVG(shrinkage))            AS stddev_shrinkage
        FROM weekly_shrink
        GROUP BY store_id
    )
    SELECT 
        ws.fiscal_week,
        ws.year,
        ws.store_name,
        ws.region,
        ROUND(ws.shrinkage, 2)                               AS shrinkage_£,
        ROUND(ss.avg_shrinkage, 2)                           AS store_baseline_£,
        ROUND(ws.shrinkage - ss.avg_shrinkage, 2)           AS variance_£,
        ROUND((ws.shrinkage - ss.avg_shrinkage) / 
              NULLIF(ss.stddev_shrinkage, 0), 2)            AS z_score,
        CASE 
            WHEN (ws.shrinkage - ss.avg_shrinkage) / 
                 NULLIF(ss.stddev_shrinkage, 0) >= 3.0 THEN 'CRITICAL'
            WHEN (ws.shrinkage - ss.avg_shrinkage) / 
                 NULLIF(ss.stddev_shrinkage, 0) >= 2.5 THEN 'HIGH ALERT'
            WHEN (ws.shrinkage - ss.avg_shrinkage) / 
                 NULLIF(ss.stddev_shrinkage, 0) >= 2.0 THEN 'ELEVATED'
            ELSE 'Normal'
        END AS alert_level
    FROM weekly_shrink ws
    JOIN store_stats ss ON ws.store_id = ss.store_id
    WHERE (ws.shrinkage - ss.avg_shrinkage) / 
          NULLIF(ss.stddev_shrinkage, 0) >= 2.0
    ORDER BY z_score DESC
""")

print(f"Total anomalies detected: {len(result)}")
print(f"CRITICAL:   {len(result[result['alert_level']=='CRITICAL'])}")
print(f"HIGH ALERT: {len(result[result['alert_level']=='HIGH ALERT'])}")
print(f"ELEVATED:   {len(result[result['alert_level']=='ELEVATED'])}")
print()
result

Total anomalies detected: 7
CRITICAL:   5
HIGH ALERT: 1
ELEVATED:   1



,fiscal_week,year,store_name,region,shrinkage_£,store_baseline_£,variance_£,z_score,alert_level
0,20,2025,Manchester Central,North,43412.56,10271.89,33140.67,6.68,CRITICAL
1,20,2024,Manchester Central,North,43049.10,10271.89,32777.21,6.60,CRITICAL
2,49,2024,Cardiff Central,Wales,15886.06,5263.93,10622.13,6.47,CRITICAL
3,51,2024,Cardiff Central,Wales,11256.95,5263.93,5993.02,3.65,CRITICAL
4,50,2024,Cardiff Central,Wales,10680.17,5263.93,5416.24,3.30,CRITICAL
5,52,2024,Cardiff Central,Wales,9696.62,5263.93,4432.69,2.70,HIGH ALERT
6,48,2024,Cardiff Central,Wales,9283.55,5263.93,4019.62,2.45,ELEVATED
